# GitHubリポジトリの準備
---

教材のバックアップにGitHubを利用する場合の、GitHubリポジトリ作成・操作に関わる手順です。

## GitHubで新しいリポジトリを作成

### リポジトリの作成
次の手順にしたがって、バックアップ用のリポジトリを作成して下さい。

1. GitHubにログインします。
  * GitHubアカウントをお持ちでない場合は作成して下さい。
2. 画面上のメニューから[Repositories]を選択します。
3. 画面右上の[New]ボタンをクリックします。下記の要領で入力します。
  * Repository name: 作成するリポジトリの名前を入力します。（必須）
  * Description: リポジトリ関する説明を入力します。（任意）
  * Public/Private: 作成したリポジトリを公開する場合は「Public」、非公開とする場合は「Private」を選択します。「Private」は有償契約の場合のみ利用できます。
  * Initialize this repository with a README: `README` を作成する場合はチェックします。`README` は後から作成できます。
  * Add .gitignore: [None]を選択。
  * Add a license: [None]を選択。
4. [Create repository]ボタンをクリックします。

## SSHキーの登録

次の手順にしたがって、リポジトリにアクセスするためのSSHキーを登録して下さい。既に登録済みのキーをお持ちに場合は、そちらをお使い頂いても構いません。

登録済みのキーを使用する場合には、`$HOME/.ssh/id_rsa` に `mode=0600` で秘密鍵を格納してください。

### SSHキーの生成

**次のセルを実行して、SSHキーペアを作成して下さい。キーペアは `$HOME/.ssh/` 以下に作成されます**
- 秘密鍵： `$HOME/.ssh/id_rsa`
- 公開鍵： `$HOME/.ssh/id_rsa.pub`

In [ ]:
%%bash
export LANG=
EMAIL="your-email@example.com"
# rm ~/.ssh/id_rsa{,.pub}
if [ -f ~/.ssh/id_rsa ]; then
  echo 'Private key already exists.' 1>&2
else
  ssh-keygen -t rsa -b 4096 -P "" -C "${EMAIL}" -f ~/.ssh/id_rsa
  echo 'Key pair generated.'
fi

**次のセルを実行して、作成した公開鍵を表示してください。コピーして GitHub に登録します。**

In [ ]:
%pycat ~/.ssh/id_rsa.pub

### GitHubに生成したキーを登録

1. GitHubにログインします。
2. 画面右上の[View profile and more]プルダウンから、[Settings]を選択します。
3. 画面左のメニューから[SSH and GPG keys]を選択します。「SSH Keys」画面が表示されます。
4. 画面右上の[New SSH key]ボタンをクリックします。
5. キー登録フォームが表示されますので、下記の要領で入力して下さい。
  * Title: キーの名前を入力します。
  * Key: 「2.1 SSHキーの生成」作成したキーペアの「公開鍵」をコピー&ペーストします。
3. [Add SSH key]ボタンをクリックします。

# GitHubリポジトリ接続情報の設定

## 接続情報の入力

* バックアップ先のGitHubリポジトリへの接続情報を設定します。

  + GIT_REPO: GitHubリポジトリのcloneパス。
  ```
   GIT_REPO=git@github.com:****/course-a.git
  ```
  + GIT_USER_NAME: GitHubのユーザ名
  ```
  GIT_USER_NAME=user
  ```
  + GIT_USER_EMAIL: コミットログに記載されるe-mailアドレス
  ```
  GIT_USER_EMAIL=user@example.com
  ```
 
**次のセルで接続情報パラメータを設定して、セルを実行して下さい。**

In [ ]:
%env GIT_REPO=git@github.com:****/course-a.git
%env GIT_USER_NAME=UserName
%env GIT_USER_EMAIL=user@example.com

## 接続情報の設定

**次のセルを実行して、GitHubリポジトリの接続情報を設定して下さい。**

In [ ]:
%%bash

cd

comm -13 <(sort ~/.ssh/known_hosts) <(ssh-keyscan github.com 2>/dev/null | sort) >>~/.ssh/known_hosts 

IS_INSIDE_REPO=0
S="$(git rev-parse --is-inside-work-tree 2>/dev/null)"
[ $? -eq 0 -a "${S}" = "true" ] && IS_INSIDE_REPO=1

if [ ${IS_INSIDE_REPO} -ne 1 ]; then
  git init
fi

git remote get-url origin >/dev/null 2>&1
if [ $? -eq 0 ]; then
  git remote rm origin

fi
git remote add origin "${GIT_REPO}"

git config user.name "${GIT_USER_NAME}"
git config user.email "${GIT_USER_EMAIL}"
git config push.default simple

git fetch --force origin

git status